<a href="https://colab.research.google.com/github/KashinathJ/Resume_Job_matcher/blob/main/Resume_job_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import os
import fitz  # PyMuPDF for PDF parsing
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from datasets import load_dataset
import numpy as np


# Specify the path to the Kaggle Resume Dataset and job descriptions from Hugging Face
hugging_face_dataset = load_dataset("jacob-hugging-face/job-descriptions")
job_descriptions = hugging_face_dataset['train']['job_description'] #JD of jobs
job_company =  hugging_face_dataset['train']['company_name'] #company names
job_role = hugging_face_dataset['train']['position_title'] #position

# Initialize DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

# Job Description Data sclidin
job_descriptions1 = job_descriptions[:15]  #  10-15 job descriptions

# Candidate-Job Matching - cosine similarity
def calculate_cosine_similarity(embeddings_1, embeddings_2):
    return cosine_similarity(embeddings_1, embeddings_2)

# Process job descriptions and store their embeddings
job_desc_embeddings = []
for job_desc in job_descriptions:
    inputs = tokenizer(job_desc, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    job_desc_embeddings.append(embeddings)


# Read the resumes from the CSV file
resume_df = pd.read_csv("/content/Resume.csv")
resume_texts = resume_df['Resume_str'] #complete details

resume_category = resume_df['Category'] #category

resume_texts1 = resume_texts#5 resumes

print(resume_texts[1])
print(type(resume_texts))
words = resume_texts.split()
for i in range(len(words)):
    if words[i] == "Education":
        extracted_name = words[i:] if i + 1 < len(words) else None
        break

sk_edu = ' '.join(extracted_name)
print(sk_edu)

#extracting education
edu_words = sk_edu.split()
for i in range(len(edu_words)):
    if edu_words[i] == "Skills":
        edu_extract = edu_words[:i] if i + 1 < len(edu_words) else None
        break

Education = ' '.join(edu_extract)
print(Education)

#extracting skills
edu_words = sk_edu.split()
for i in range(len(edu_words)):
    if edu_words[i] == "Skills":
        edu_extract = edu_words[i:] if i + 1 < len(edu_words) else None
        break

Skills = ' '.join(edu_extract)
print(Skills)

# Process resumes and match them with job descriptions
resume_matches = {}
for i, resume_text in enumerate(resume_texts):
    inputs = tokenizer(resume_text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    resume_embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()

    #  cosine similarity with job descriptions
    similarity_scores = [cosine_similarity(job_emb.reshape(1, -1), resume_embeddings.reshape(1, -1))[0][0] for job_emb in job_desc_embeddings]

    # Find the index of the top match
    top_match_index = np.argmax(similarity_scores)

    # Store the top match for each resume
    resume_matches[resume_text] = (job_company[top_match_index], job_role[top_match_index], job_descriptions1[top_match_index], similarity_scores[top_match_index] )

# Print the top match for each resume
for resume_text,  (comp,  role, job_desc,score) in resume_matches.items():
    print(f"Resume: {resume_text[:100]}...\nCompany:{comp}...\nRole:{role} \nJob Description: {job_desc[:100]}... \nSimilarity Score: {score:.2f}\n")
